# Порождающая RNN

Эта тетрадка основана на 6 главе Николенко. Пришло время обучить на каком-нибудь корпусе текстов порождающую нейронную сетку. Для того, чтобы делать это, нужно переработать тексты в приемлимые для работы датасеты. Идея, находящаяся под капотом очень проста: __давайте будем порождать текст буква за буквой,__ рассматривая его как поток символов. Тогда задача очень легко формализуется. Нужно предсказать следущий символ по текущему, то есть на каждом шаге нужно по накопленной истории решать задачу классификации на десяток классов.

Вопрос в том, как для такой модели определить тренировочные данные.

* __Первый, самый простой вариант__ - взять текст и разбить его на последовательности фиксированной длинны (окна) и предсказывать следущий по предыдущим. В такой ситуации начало и конец тренировочного примера будут довольно "внезапными", и артефакты по краям будут мешать генерировать на выходе хороший текст.
* __Второй вариант__ - дробить текст на последовательности разной длины, но не слишком большой. Например, на предложения. Затем мы добавим спецсимволы начала и конца предложения и дополним каждую строчку до ммаксимума спецсимволом, означающим пустоту.
* __Третий вариант__ - нарезать текст на последовательности примерно одной длины, но при этом правильным образом инициализировать скрытые состояния сетки. Для этого нужно взять наш длинный-длинный вход и превратить его сначала в достаточно широкий прямоугольник размера $N \times L$, где $L$ - длина каждого тренировочного примера, а $N$ - число тренировочных примеров в исходной последовательности. Пока что $L$ очень велика. Но мы можем разрезать прямоугольник на более маленькие батчи по вертикали. Получиться, что каждый новый батч - продолжение предыдущего и его нужно обучать не с нуля, а с того скрытого состояния, на котором закончился предыдущий батч. 

Мы будем использовать второй вариант и просто дробить текст на предложения. 

## 1. Предобработка выборки

Будем строить модельна примери Евгения Онегина. Считаем данные, определим три специальных символа: `START_CHAR` будет подставляться перед началом предложеия, `END_CHAR` после его конца, а `PADDING_CHAR` будет заполнять остаток предложения до максимума длины. Для простоты мы также не будетм различать строчные и заглавные буквы, а просто пропустим каждую строчку через `lower()`.

In [3]:
START_CHAR = '\b'
END_CHAR = '\t'
PADDING_CHAR = '\a'

chars = set([START_CHAR, '\n', END_CHAR])

with open('onegin.txt') as f:
    for line in f:
        chars.update(list(line.strip().lower()))
    
# заведём словари с отображением символов в числа и обратно    
char_indices = { c : i for i,c in enumerate(sorted(list(chars))) }
char_indices[PADDING_CHAR] = 0
indices_to_chars = { i : c for c,i in char_indices.items() }
num_chars = len(chars)

num_chars # уникальные символы

75

Теперь создадим векторные представления для символов. Это будет просто OHE представления, в котором каждому символу соответствует вектор с одной единицей, за исключением PADDING_CHAR, который будет представляться просто нулевым вектором. 

In [4]:
import numpy as np

def get_one(i, sz):
    res = np.zeros(sz)
    res[i] = 1
    return res

char_vectors = {
    c : (np.zeros(num_chars) if c == PADDING_CHAR else get_one(v, num_chars))
    for c,v in char_indices.items()
}

Теперь считаем исходный файл ещё разок. На этот раз разобьём его на предложения. Будем брать только предложения длиннее 10 символов. 

In [5]:
sentence_end_markers = set('.!?')

sentences = [ ]
current_sentence = ''
with open('onegin.txt', 'r') as f:
    for line in f:
        s = line.strip().lower()
        if len(s) > 0:
            current_sentence += s + '\n'
        if len(s) == 0 or s[-1] in sentence_end_markers:
            current_sentence = current_sentence.strip()
            if len(current_sentence) > 10:
                sentences.append(current_sentence)
            current_sentence = ''
            
sentences[0]

'не мысля гордый свет забавить,\nвниманье дружбы возлюбя,\nхотел бы я тебе представить\nзалог достойнее тебя,\nдостойнее души прекрасной,\nсвятой исполненной мечты,\nпоэзии живой и ясной,\nвысоких дум и простоты;\nно так и быть — рукой пристрастной\nприми собранье пестрых глав,\nполусмешных, полупечальных,\nпростонародных, идеальных,\nнебрежный плод моих забав,\nбессониц, легких вдохновений,\nнезрелых и увядших лет,\nума холодных наблюдений\nи сердца горестных замет.'

Следущий шаг - векторизация. Давайте определим процедуру, которая превращает набор предложений в два тензора: $X$ содержит векторы символов, а $Y$ результат, который нам нужно предсказать. Это на самом деле ровно тот же тензор $X$, только сдвинутый на один вектор влево: во время $t$ мы предсказываем символ, который будет стоять на месте $t+1$. 

In [6]:
def get_matrices(sentences):
    max_sentence_len = np.max([len(x) for x in sentences])
    X = np.zeros((len(sentences), max_sentence_len, len(chars)), dtype = np.bool)
    Y = np.zeros((len(sentences), max_sentence_len, len(chars)), dtype = np.bool)
    for i, sentence in enumerate(sentences):
        char_seq = (START_CHAR + sentence + END_CHAR).ljust(
                      max_sentence_len + 1, PADDING_CHAR)
        for t in range(max_sentence_len):
            X[i, t, :] = char_vectors[char_seq[t]]
            Y[i, t, :] = char_vectors[char_seq[t+1]]
    return X,Y

Для экономии памяти и ускорения мы вместо единичек и нулей ставим булевы значеия TRUE и FALSE. Обратите внимание, что мы в начало каждого предложения и в его конец добавили особые символы и дополнили каждое из ни пустотами до длины максимального предложения функцией `ljust`. Наконец мы готовы к строительству нейросетки.

## 2. Архитектура

Построим простую модель. Добавим один уровень LSTM ячеек, результат работы которых будем пропускать через один полносвязный слой, на котором и будет происходить классификация. 

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, TimeDistributed, Activation

model = Sequential()
model.add(LSTM(output_dim = 128, activation = 'tanh', 
               return_sequences = True, input_dim = num_chars))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_dim = num_chars)))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, input_shape=(None, 75), units=128)`
  


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=75)`
  


Ура! Архитектура собралась. Прокомментируем её. 

* Параметр `output_dim = 128` означает, что наш слой состоит из 128 LSTM-ячеек, на этом слое в качестве функции активации используем тангенс. Можно при желании поменять на сигмоиду... 
* Параметр `input_dim = X.shape[2]` задаёт размерность входа, то есть число символов. В нашем случае это `len(chars)`.
* Параметр `return_sequences = True` здесь самый интересный. По умолчанию LSTM-ячейка идёт по входной последовательности, в нашем случае предложению, и на выход выдаёт только самый последний рещультат. Мы бы хотели, чтобы она сохраняла всю последовательность по мере продвижения.

Слой `TimeDistributed` тоже является для нас новой штукой. Его фишка в том, что веса полносвязного словя не меняются во времени. Для всех выходов из LSTM всегда используются одни и те же веса. 

Вся наша собранная модель нарисована на картинке ниже: 

Модель осталось скомпилировать и начать процедуру оптимизации. Как вы помните, рекурентные нейронки страдают проблемой взрыва градиентов. Чтобы этой проблемы избежать, их купируют (обрезают). Для этого используют два параметра: 

* `clipnorm` будет масштабировать вектор градиента так, чтобы его норма не превысила заданного порога
* `clipvalue` будет просто обрезать до заданного порога каждую компоненту градиента по отдельности

In [8]:
from keras.optimizers import Adam
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(clipnorm = 1.), metrics = ['accuracy'])

Мы почти готовы к обучению. Остались две маленькие детали. Когда мы писали функцию `get_matrices`, мы оформили её так, что размерность тензоров на выходе зависит от максимальной длины предложений на входе. Мы сделали так специально, чтобы подавать в нашу нейросетку данные батчами. Нужно обернуть функцию `get_matrices` в генератор.

In [9]:
# выделим тестовую выборку
test_indices = np.random.choice(range(len(sentences)), int(len(sentences) * 0.05))

sentences_train = [sentences[x] for x in set(range(len(sentences))) - set(test_indices)]
sentences_test = [sentences[x] for x in test_indices]

sentences_train = sorted(sentences_train, key = lambda x : len(x))
X_test, y_test = get_matrices(sentences_test)

batch_size = 16
def generate_batch():
    while True:
        for i in range( int(len(sentences_train)/batch_size)):
            sentences_batch = sentences_train[i*batch_size : (i+1)*batch_size]
            yield get_matrices(sentences_batch)

Теперь функция `generate_batch` последовательно будет проходить наш датасет, разбивая его на мини-батчи по 16 предложений. А чтобы это имело ещё больше смысла мы предварительно отсортировали `sentences_train` по длине, так что теперь мини-батчи будут иметь последовательно увеличивающуюся ширину, предложения в них будут иметь примерно одинаковую длину и заполнять `PADDING_CHAR` придется не так много остатков. 

В ходе обучения модели нам часто хочется наблюдать за какими-нибудь её особенностями. В этом обычно помогают специальные функции: функции обратного вызова (callbacks). Если в этот метод передать список функций, то они будут запускаться после каждой эпохи обучения. И мы сможем отслеживать интересующую нас при обучении сетки статистику. Давайте добавим две стандартные функции обратного вызова и напишем одну свою. 

Сразу же добавляем свою. Как вы могли бы в с помнить, на самой первой паре мы говорили о Softmax. Мы говорили, что это мягкий максимум. Он при трансформации полученных для каждой категории чисел в вероятности, пытается приподнять самое большое число и приопустить маленькие, мягко выделяя максимум. 

Силу такого выделения можно контролировать. Для этого в формулу вшивается дополнительный параметр $T$, который называют температурой сэмплирования (см пример на странице 267 Николенко). 

$$ p(w) \propto e^{-\frac{1}{T} x_{w}} $$

Если взять $T$ очень большим, то показатели экспоненты окажутся небольшими по модулю и возведения в степень будут близки. На выходе получится близкое к равномерному распределение. Сэмплирование будет довольно случайным. Взвинчивание $T$ позволяет делать сэмплирование конкретным. При $T \to 0$ распределение будет вырожденным. 

Напишем колбэк, который будет подставлять в сетку разные значения температуры при генерации. 

In [10]:
import os
from keras.callbacks import Callback

class CharSampler(Callback):
    def __init__(self, char_vectors, model):
        self.char_vectors = char_vectors
        self.model = model 
        
    def on_train_begin(self, logs={}):
        self.epoch = 0
        if os.path.isfile('output_file'):
            os.remove('output_file')
            
    def sample(self, preds, temperature=1.0):
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds)/temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    def sample_one(self, T):
        result = START_CHAR
        while len(result) < 500:
            Xsampled = np.zeros((1, len(result), num_chars))
            for t,c in enumerate(list(result)):
                Xsampled[0, t, :] = self.char_vectors[c]
            ysampled = self.model.predict(Xsampled, batch_size=1)[0,:]
            yv = ysampled[len(result) - 1, :]
            selected_char = indices_to_chars[self.sample(yv, T)]
            if selected_char == END_CHAR:
                break
            result = result + selected_char
        return result

    def on_epoch_end(self, batch, logs = {}):
        self.epoch_end = self.epoch + 1
        if self.epoch % 50 == 0:
            print("\nEpoch %d text sampling:" % self.epoch)
            with open('output_file', 'a') as outf:
                outf.write('\n======= Epoch %d =======\n' % self.epoch)
                for T in [0.3, 0.5, 0.7, 0.9, 1.1]:
                    print('\tsampling, T = %.1f...' % T)
                    for _ in range(5):
                        self.model.reset_states()
                        res = self.sample_one(T)
                        outf.write('\nT = %.1f\n%s\n' % (T, res[1:]))
                        

Добавим в сетку ещё один колбэк для логирования результатов. 

In [13]:
from keras.callbacks import CSVLogger
cb_logger = CSVLogger('sim/' + 'onegin' + '.log')
cb_sampler = CharSampler(char_vectors, model)

Наконец мы готовы учить. Для обучения по батчам испольщуем `fit_generate`.

In [14]:
model.fit_generator(generate_batch(),
                     int(len(sentences_train) / batch_size) * batch_size,
                     nb_epoch=1000,
                     verbose=True, 
                     validation_data = (X_test, y_test), 
                     callbacks=[cb_logger, cb_sampler])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 1136, verbose=True, validation_data=(array([[[..., callbacks=[<keras.ca..., epochs=1000)`
  


Epoch 1/1000
1136/1136 [==============================] - 120s 106ms/step - loss: 2.7275 - acc: 0.2185 - val_loss: 0.9167 - val_acc: 0.1019

Epoch 0 text sampling:
	sampling, T = 0.3...
	sampling, T = 0.5...
	sampling, T = 0.7...
	sampling, T = 0.9...
	sampling, T = 1.1...
Epoch 2/1000
1136/1136 [==============================] - 117s 103ms/step - loss: 2.3652 - acc: 0.2812 - val_loss: 0.8689 - val_acc: 0.1139

Epoch 0 text sampling:
	sampling, T = 0.3...
	sampling, T = 0.5...
	sampling, T = 0.7...
	sampling, T = 0.9...
	sampling, T = 1.1...
Epoch 3/1000
 754/1136 [==================>...........] - ETA: 37s - loss: 2.2554 - acc: 0.3108

KeyboardInterrupt: 